In [2]:
import torch

In [3]:
import pandas as pd
import os

In [4]:
annotations_files = ['fold_0_data.txt','fold_1_data.txt','fold_2_data.txt','fold_3_data.txt','fold_4_data.txt']
data = pd.concat([pd.read_csv(i, delimiter='\t') for i in annotations_files])
data.shape

(19370, 12)

In [5]:
data['gender'].unique()

array(['f', 'm', nan, 'u'], dtype=object)

In [6]:
g = data['gender']
data = data[(g=='f')|(g=='m')]
data['gender'] = data['gender'].map(lambda x: (int)(x=='f'))
data.shape

(17492, 12)

In [7]:
img_paths = []
for i in data.iterrows():
    img_paths.append(os.path.join(i[1]['user_id'], i[1]['original_image']))
data['img_path'] = img_paths

In [8]:
data['age'].value_counts()

(25, 32)     4951
(38, 43)     2293
(4, 6)       2139
(8, 12)      2117
(15, 20)     1640
(0, 2)       1397
(60, 100)     866
(48, 53)      825
35            293
13            168
22            149
34            105
23             96
45             88
(27, 32)       77
55             76
36             56
(38, 42)       46
None           40
57             24
3              18
29             11
(38, 48)        6
58              5
2               3
46              1
(8, 23)         1
42              1
Name: age, dtype: int64

In [9]:
ag = [(0,3),(4,7),(8,14),(15,20),(21,32),(33,43),(44,59),(60,100)]

In [10]:
age_embedding, is_baby = [], []
for i in data.iterrows():
    a = i[1]['age']
    try:
        a = int(a)
    except:
        pass
    if a == 'None':
        res = 0
    else:
        if type(a) is not int:
            a = int(a[1:-1].split(',')[0])
        for x,y in enumerate(ag):
            if a >= y[0] and a <= y[1]:
                res = x + 1
                break
    age_embedding.append(res)
    is_baby.append(int(res==1))
    
data['age_embedding'], data['is_baby'] = age_embedding, is_baby

In [11]:
data['age_embedding'].value_counts()

5    5284
6    2800
3    2286
2    2139
4    1640
1    1418
7    1019
8     866
0      40
Name: age_embedding, dtype: int64

In [12]:
data['is_baby'].value_counts()

0    16074
1     1418
Name: is_baby, dtype: int64

In [13]:
train = pd.read_csv("X_train.csv")
test = pd.read_csv("X_test.csv")

In [15]:
age_embedding, is_baby = [], []
for i in train.iterrows():
    img = i[1]['img_path']
    row = data.loc[data['img_path']==img]
    age_embedding.append(row.iloc[0]['age_embedding'])
    is_baby.append(row.iloc[0]['is_baby'])

In [16]:
train['age_embed'], train['is_baby'] = age_embedding, is_baby

In [17]:
train['age_embed'].value_counts()

5    4501
6    2533
3    1727
4    1348
2    1326
1     933
7     866
8     735
0      24
Name: age_embed, dtype: int64

In [18]:
age_embedding, is_baby = [], []
for i in test.iterrows():
    img = i[1]['img_path']
    row = data.loc[data['img_path']==img]
    age_embedding.append(row.iloc[0]['age_embedding'])
    is_baby.append(row.iloc[0]['is_baby'])
test['age_embed'], test['is_baby'] = age_embedding, is_baby

In [19]:
test['age_embed'].value_counts()

5    1153
6     599
3     392
2     351
4     327
1     241
7     228
8     197
0      11
Name: age_embed, dtype: int64

In [20]:
train.to_csv('X_train.csv', index=False)
test.to_csv('X_test.csv', index=False)